In [38]:
import json
import gzip
import pandas as pd

out_dir = "parse_parquet"
chunk = 100000
index = 0

data = []
with gzip.open("enwiki-20230918-cirrussearch-content.json.gz") as f:
    for line in f:
        json_line = json.loads(line)
        if "index" not in json_line:
            data.append(json_line)
            if len(data) >= chunk:
                pd.DataFrame(data)[["title", "text", "page_id", "category"]].to_parquet(f"{out_dir}/{index}.parquet")
                data = []
                index += 1


In [3]:
import glob
import json
import gzip
import pandas as pd


In [4]:
df = pd.concat([
    pd.read_parquet(f) for f in glob.glob("parse_parquet/*.parquet")
])

In [5]:
df

,title,text,page_id,category
0,Hercule Poirot,"Hercule Poirot (UK: /ˈɛərkjuːl ˈpwɑːroʊ/, US: ...",1000,[Wikipedia articles needing page number citati...
1,Eiffel,"Look up Eiffel in Wiktionary, the free diction...",10000,"[Short description is different from Wikidata,..."
2,Juan que reía,Juan que reía is a 1976 Argentine film. Luis B...,10000001,[Articles needing additional references from M...
3,La Noche del hurto,La Noche del hurto is a 1976 Argentine comedy ...,10000009,[Articles needing additional references from A...
4,NPU,"Look up npu in Wiktionary, the free dictionary...",1000001,"[Short description is different from Wikidata,..."
...,...,...,...,...
99995,Adam Brooks (filmmaker),"Adam Brooks (born September 3, 1956) is a Cana...",18509046,"[Articles with short description, Short descri..."
99996,Yamada Station (Gifu),"Yamada Station (山田駅, Yamada-eki) is a railway ...",18509048,"[CS1 uses Japanese-language script (ja), CS1 J..."
99997,Tokunaga Station,"Tokunaga Station (徳永駅, Tokunaga-eki) is a rail...",18509060,"[CS1 uses Japanese-language script (ja), CS1 J..."
99998,Wirth Lake (Minnesota),"Wirth Lake is in Theodore Wirth Park, a large ...",1850907,"[Articles with short description, Short descri..."


In [6]:
df["first"] = df["title"].apply(lambda x: x[0].lower())

In [7]:
df["alpha"] = df["first"].apply(lambda x: x.isalpha())

In [8]:
df = df.reset_index(drop=True)

In [9]:
df["id"] = df["page_id"].astype(str)

In [11]:
alphabets = list("abcdefghijklmnopqrstuvwxyz")
len(alphabets)

26

In [12]:
for alphabet in alphabets:
    w_df = df[df["first"] == alphabet]
    w_df[["title", "text", "id", "category"]].to_parquet(f"../data/wikipedia_fixed/{alphabet}.parquet")

In [13]:
df[~df["first"].isin(alphabets)][["title", "text", "id", "category"]].to_parquet("../data/wikipedia_fixed/other.parquet")